In [23]:
import codemaster as cm
import numpy as np
import random
import gensim
from nltk.corpus import words
from functools import reduce
import heapq


In [76]:
wrds = np.genfromtxt('wordlist.csv', delimiter=',', dtype=str).tolist()

# randomly select 25 cards, then randomly select a side for each
board = [x[random.random() > 0.5].lower() for x in random.sample(wrds, 25)]

# set up game by assigning each card (agent) a role
player1 = board[:9]
player2 = board[9:17]
neutral = board[17:24]
assassin = board[24]


In [24]:

# load model pre-trained on Google News corpus (downloaded from https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit)
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

# choose candidate clues from nltk's words corpus, which itself is drawn from the UNIX words file
dct = words.words('en')


In [64]:
# returns potential clues which are closer to all members of targets than all members of avoids, sorted in order of relevance to targets
def clue (targets, avoids, similarity_f, score_f, n=1):
    topclues = []
    for word in dct:
        if word not in model.vocab:
            continue
        if avoids:
            target_distances = [similarity_f(word, target) for target in targets]
            avoid_distances = [similarity_f(word, avoid) for avoid in avoids]
            if max(avoid_distances) > min(target_distances):
                continue
        score = score_f([similarity_f(word, target) for target in targets])
        if reduce((lambda x, y: y not in word and word not in y and x), targets, True):
            if len(topclues) < n:
                heapq.heappush(topclues, (score, word))
            elif score > topclues[0][0]:
                heapq.heapreplace(topclues, (score, word))
    return sorted(topclues, key=lambda x: x[0], reverse=True)

In [36]:
def model_sim(word, target):
    return model.similarity(word, target)


In [51]:
candidates = clue(board[:2], [], model_sim, sum, n=5)
candidates_1 = clue(board[:2], [], model_sim, min, n=5)
print ("SUM")
print ('Top clues for pair', board[:2], ':', candidates)
print ("MIN")
print ('Top clues for pair', board[:2], ':', candidates_1)


D:\anaconda\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


SUM
Top clues for pair ['shoe', 'fire'] : [(0.8120177462697029, 'blaze'), (0.7323597595095634, 'footwear'), (0.7172053307294846, 'sandal'), (0.6636006273329258, 'sneaker'), (0.6607478559017181, 'clothing')]
MIN
Top clues for pair ['shoe', 'fire'] : [(0.25620458, 'candle'), (0.25490576, 'bunkie'), (0.25222316, 'haymow'), (0.25202337, 'snuffer'), (0.24360001, 'sparkler')]


In [77]:
clues = []

for i in range(len(player2)):
    print (clues)
    for j in range(i+1,len(player2)):
        targets = []
        targets.append(player2[i])
        targets.append(player2[j])
        candidates = clue(targets, player1, model_sim, min, n=5)
        for candidate in candidates:
            if len(clues) < 15:
                heapq.heappush(clues, (candidate[0], candidate[1], targets[0], targets[1]))
            elif candidate[0] > clues[0][0]:
                heapq.heapreplace(clues, (candidate[0], candidate[1], targets[0], targets[1]))

[]


D:\anaconda\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[(0.26510388, 'outpass', 'embassy', 'ball'), (0.26766312, 'cabled', 'embassy', 'satellite'), (0.2711347, 'spy', 'embassy', 'alien'), (0.26921424, 'subversive', 'embassy', 'alien'), (0.27060372, 'palace', 'embassy', 'mint'), (0.3068303, 'delegation', 'embassy', 'trip'), (0.28232914, 'grouplet', 'embassy', 'alien'), (0.32086203, 'emplane', 'embassy', 'trip'), (0.3379429, 'villa', 'embassy', 'compound'), (0.28923923, 'spy', 'embassy', 'satellite'), (0.39866394, 'guesthouse', 'embassy', 'compound'), (0.308164, 'mosque', 'embassy', 'compound'), (0.36451837, 'palace', 'embassy', 'compound'), (0.33849293, 'residence', 'embassy', 'compound'), (0.31807834, 'foreigner', 'embassy', 'alien')]
[(0.28088975, 'adventure', 'trip', 'alien'), (0.28390276, 'tourney', 'trip', 'ball'), (0.28232914, 'grouplet', 'embassy', 'alien'), (0.31688452, 'tournament', 'trip', 'ball'), (0.28923923, 'spy', 'embassy', 'satellite'), (0.3068303, 'delegation', 'embassy', 'trip'), (0.29317012, 'visitor', 'trip', 'pupil'), (

In [78]:
clues_sorted = sorted(clues, key=lambda x: x[0], reverse=False)
for c in clues_sorted:
    print (c)
print (player2)
print (player1)

(0.3068303, 'delegation', 'embassy', 'trip')
(0.308164, 'mosque', 'embassy', 'compound')
(0.31374758, 'limonene', 'mint', 'compound')
(0.31688452, 'tournament', 'trip', 'ball')
(0.31807834, 'foreigner', 'embassy', 'alien')
(0.3201443, 'mothership', 'satellite', 'alien')
(0.32086203, 'emplane', 'embassy', 'trip')
(0.32111037, 'unpiloted', 'satellite', 'alien')
(0.32611704, 'lunar', 'satellite', 'alien')
(0.33073822, 'extrasolar', 'satellite', 'alien')
(0.3379429, 'villa', 'embassy', 'compound')
(0.33849293, 'residence', 'embassy', 'compound')
(0.3458283, 'interplanetary', 'satellite', 'alien')
(0.36451837, 'palace', 'embassy', 'compound')
(0.39866394, 'guesthouse', 'embassy', 'compound')
['embassy', 'trip', 'ball', 'mint', 'satellite', 'alien', 'pupil', 'compound']
['plane', 'state', 'cap', 'octopus', 'organ', 'van', 'thumb', 'ray', 'court']
